# Aircraft Type Trends Over Time

This notebook charts aicraft models over time, producing monthly and yearly charts.

Individual models within a family are grouped:

- e.g. Both A320-214 and A320-232 will appear under the "A320" family
- Aircraft with fewer sightings than the configured threshold are assigned to the "Other" family

In [1]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

def export_monthly_trends_chart(export_folder_path, data, number_of_months):
    # Create a new figure
    fig, ax = plt.subplots(figsize=(14, 7))

    # Create the plot
    data.plot(
        kind="bar",
        stacked=True,
        colormap="tab20",
        ax=ax
    )

    # Set X-axis label formatting
    ax.set_xticklabels([d.strftime('%Y-%m') for d in data.index])

    # Set chart titles
    plt.title(f"Monthly Sightings by Aircraft Type ({number_of_months} Months)")
    plt.xlabel("")
    plt.ylabel("Sightings")
    plt.xticks(rotation=90)

    # Move legend below the plot
    plt.legend(
        title="Aircraft Type",
        bbox_to_anchor=(0.5, -0.2),
        loc='upper center',
        ncol=4,
        frameon=False
    )

    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, "aircraft-type-trends-monthly", "png")

    # Close the plot
    plt.close(fig)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

def export_yearly_trends_chart(export_folder_path, data):
    # Create a new figure
    fig, ax = plt.subplots(figsize=(14, 7))

    # Create the plot
    data.plot(
        kind="bar",
        stacked=True,
        colormap="tab20",
        ax=ax
    )

    # Set chart titles
    plt.title("Yearly Sightings by Aircraft Type")
    plt.xlabel("")
    plt.ylabel("Sightings")
    plt.xticks(rotation=90)

    # Move legend below the plot
    plt.legend(
        title="Aircraft Type",
        bbox_to_anchor=(0.5, -0.2),
        loc='upper center',
        ncol=4,
        frameon=False
    )

    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, "aircraft-type-trends-yearly", "png")

    # Close the plot
    plt.close(fig)

In [4]:
# Sightings threshold below which aircraft are assigned to a generic "Other" group
prefix_count_threshold = 100

# Number of months to represent in the monthly chart
number_of_months = 12

In [5]:
import pandas as pd

# Construct the query
query = construct_query("aircraft", "sightings.sql", {})

try:
    # Run the query to load the data - this raises a ValueError if there's no data
    df = query_data("aircraft", query)

    # Extract an aircraft type prefix for each sighting
    df["Type"] = df["Model"].apply(extract_prefix)

    # Add a "year/month" column to the data for monthly reporting
    df["Year_Month"] = df["Date"].dt.to_period("M")

    # Identify counts by prefix and use them to identify common prefixes
    prefix_counts = df["Type"].value_counts()
    common_prefixes = prefix_counts[prefix_counts >= prefix_count_threshold].index

    # Replace uncommon prefixes with "Other"
    df["Type"] = df.apply(
        lambda row: row["Type"] if row["Type"] in common_prefixes else "Other",
        axis=1
    )

    # Generate the monthly trend data by aircraft type
    monthly_counts = (
        df.groupby(["Year_Month", "Type"])
        .size()
        .unstack(fill_value=0)
    )

    # Convert PeriodIndex to datetime for plotting
    monthly_counts.index = monthly_counts.index.to_timestamp()

    # Calculate the yearly trends
    df["Year"] = df["Date"].dt.year

    yearly_counts = (
        df.groupby(["Year", "Type"])
        .size()
        .unstack(fill_value=0)
    )

    # Export the data to excel
    export_folder_path = get_export_folder_path()
    export_to_spreadsheet(export_folder_path, "aircraft-type-trends-monthly.xlsx", {
        "Monthly": monthly_counts
    })
    export_to_spreadsheet(export_folder_path, "aircraft-type-trends-yearly.xlsx", {
        "Yearly": yearly_counts
    })

    # Create a copy of the monthly counts that only includes the last "N" months, for charting
    monthly_counts.index = pd.to_datetime(monthly_counts.index)
    current_month = pd.Timestamp.today().to_period("M").to_timestamp()
    start_month = current_month - pd.DateOffset(months=number_of_months-1)
    monthly_counts_last_n_months = monthly_counts.loc[start_month:current_month].copy()

    # Export the charts
    export_monthly_trends_chart(export_folder_path, monthly_counts_last_n_months, number_of_months)
    export_yearly_trends_chart(export_folder_path, yearly_counts)

except ValueError:
    print(f"WARNING: No data found")